In [1]:
import json
from functools import partial
from typing import List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from scipy.spatial.distance import cosine, euclidean
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score


### Load data

In [2]:
dataset = pd.read_parquet(r"C:\Users\druzh\Project_python\ozon_top_1\Datasets/train_pairs.parquet")
etl = pd.read_parquet(r"C:\Users\druzh\Project_python\ozon_top_1\Datasets/train_data.parquet")

In [3]:
dataset.head(2)

,target,variantid1,variantid2
0,0.0,51197862,51198054
1,1.0,53062686,536165289


Get raw data for each variantid.

In [4]:
features = (
    dataset
    .merge(
        etl
        .add_suffix('1'),
        on="variantid1"
    )
    .merge(
        etl
        .add_suffix('2'),
        on="variantid2"
    )
)

In [5]:
features.head(2)

,target,variantid1,variantid2,name1,categories1,color_parsed1,pic_embeddings_resnet_v11,main_pic_embeddings_resnet_v11,name_bert_641,characteristic_attributes_mapping1,name2,categories2,color_parsed2,pic_embeddings_resnet_v12,main_pic_embeddings_resnet_v12,name_bert_642,characteristic_attributes_mapping2
0,0.0,51197862,51198054,Удлинитель TDM Electric Люкс УЛ05В 5 м (SQ1303...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Сетевые...",[белый],None,"[[-0.4304909, -0.49474272, -0.46439183, -0.060...","[-0.5104684, 0.56158644, 0.58873796, -0.529718...","{""Число жил"":[""3""],""Макс. нагрузка, Вт"":[""3500...",Удлинитель TDM Electric Люкс УЛ05В 1.5 м (SQ13...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Сетевые...",[белый],None,"[[-0.42941108, -0.5129398, -0.4753536, -0.0677...","[-0.455473, 0.58157134, 0.5870387, -0.5325003,...","{""Электробезопасность"":[""Заземление""],""Длина к..."
1,0.0,51197862,51199884,Удлинитель TDM Electric Люкс УЛ05В 5 м (SQ1303...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Сетевые...",[белый],None,"[[-0.4304909, -0.49474272, -0.46439183, -0.060...","[-0.5104684, 0.56158644, 0.58873796, -0.529718...","{""Число жил"":[""3""],""Макс. нагрузка, Вт"":[""3500...",Удлинитель TDM Electric Люкс УЛ05В 3 м (SQ1303...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Сетевые...",[белый],None,"[[-0.43180764, -0.49580905, -0.5062628, -0.130...","[-0.5425725, 0.6415736, 0.51481575, -0.5687392...","{""Макс. нагрузка, Вт"":[""3500""],""Стандарт защит..."


Features functions.

In [10]:
feats = ["main_pic_embeddings_resnet_v11", "name_bert_641", "main_pic_embeddings_resnet_v12", "name_bert_642"]

In [11]:
X_train, X_test = train_test_split(
    features[feats + ["target", "variantid1", "variantid2"]], 
    test_size=0.1, random_state=42, stratify=features[["target"]])

X_train, X_val = train_test_split(
    X_train[feats + ["target"]], 
    test_size=0.1, random_state=42, stratify=X_train[["target"]])


y_test = X_test[["target"]]
X_test = X_test.drop(["target", "variantid1", "variantid2"], axis=1)

y_train = X_train["target"] 
X_train = X_train.drop(["target"], axis=1)

        


In [19]:
X_train.head(2)

,main_pic_embeddings_resnet_v11,name_bert_641,main_pic_embeddings_resnet_v12,name_bert_642
276475,"[[1.5177895, -0.1607113, -1.3436879, -0.518293...","[-0.5862704, 0.39754072, 0.7764767, -0.5785445...","[[1.330656, -0.2443413, -1.4133728, -0.3939227...","[-0.5026867, 0.36081815, 0.7274831, -0.5621019..."
238547,"[[0.4010295, -0.16012707, -0.9049238, -0.00406...","[-0.49965647, 0.75911903, 0.5878046, -0.476294...","[[0.19791354, 0.031553254, -0.44747415, 0.1454...","[-0.498988, 0.7581946, 0.6100502, -0.45187664,..."


In [23]:
X_train["main_pic_embeddings_resnet_v11"][276475][0]

array([ 1.5177895 , -0.1607113 , -1.3436879 , -0.5182934 ,  1.9222594 ,
        1.409966  , -0.07100956, -1.3819436 ,  0.29979265, -0.46199632,
        0.41519195,  0.62992084, -0.37708902,  0.526022  , -1.3048393 ,
       -0.3913864 ,  1.6058913 ,  0.40473   , -0.02482107, -0.573033  ,
       -0.70632994, -0.15699726, -0.8837389 ,  0.804603  ,  0.12449124,
       -0.8059559 , -0.7594    , -0.4075185 , -0.73291975,  1.1321254 ,
       -0.05538362, -0.24840048, -0.35928893,  1.7011063 , -0.34390223,
        0.2926876 , -0.39033335, -1.3694782 ,  1.3234332 , -0.3599332 ,
       -1.5499215 ,  0.09847139,  1.3356067 , -0.32096905, -0.5689085 ,
        0.00678925, -0.27474916,  1.0556518 , -1.9857645 , -0.75344956,
        1.6898749 ,  0.5296958 ,  0.15033466, -0.5294885 , -0.06423454,
        0.547827  , -1.2910435 ,  0.02101413, -0.05007964,  0.9777609 ,
        0.82468736, -0.55121124,  1.6157899 , -0.51937044,  0.5853788 ,
        0.6077431 , -0.10538843,  0.1991615 , -0.33610523, -0.66

In [24]:
X_train_final = []

for i in range(len(X_train)):
    row = []
    row.extend(list(X_train[feats[0]].iloc[i])[0])
    row.extend(list(X_train[feats[1]].iloc[i]))
    row.extend(list(X_train[feats[2]].iloc[i])[0])
    row.extend(list(X_train[feats[3]].iloc[i]))
    X_train_final.append(row)

In [27]:
X_test_final = []

for i in range(len(X_test)):
    row = []
    row.extend(list(X_test[feats[0]].iloc[i])[0])
    row.extend(list(X_test[feats[1]].iloc[i]))
    row.extend(list(X_test[feats[2]].iloc[i])[0])
    row.extend(list(X_test[feats[3]].iloc[i]))
    X_test_final.append(row)

In [28]:
from sklearn.linear_model import SGDClassifier

def train_logistic_regression(X, y, chunk_size, num_epochs=10):
    clf = SGDClassifier(loss='log')  # Используем логистическую регрессию
    num_samples = len(X)
    num_chunks = num_samples // chunk_size

    for epoch in range(num_epochs):
        print(epoch)
        for chunk_idx in range(num_chunks):
            start_idx = chunk_idx * chunk_size
            end_idx = start_idx + chunk_size
            X_chunk = X[start_idx:end_idx]
            y_chunk = y[start_idx:end_idx]
            clf.partial_fit(X_chunk, y_chunk, classes=np.unique(y))
    
    return clf

In [30]:
chunk_size = 60000
num_epochs = 200

model = train_logistic_regression(X_train_final, y_train, chunk_size, num_epochs)

0


C:\Users\druzh\AppData\Local\Temp\ipykernel_10688\132693369.py:14: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_chunk = y[start_idx:end_idx]
C:\Users\druzh\Project_python\venv\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
C:\Users\druzh\AppData\Local\Temp\ipykernel_10688\132693369.py:14: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, 

1


C:\Users\druzh\AppData\Local\Temp\ipykernel_10688\132693369.py:14: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_chunk = y[start_idx:end_idx]
C:\Users\druzh\AppData\Local\Temp\ipykernel_10688\132693369.py:14: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_chunk = y[start_idx:end_idx]
C:\Users\druzh\AppData\Local\Temp\ipykernel_10688\132693369.py:14: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated

2


...

C:\Users\druzh\AppData\Local\Temp\ipykernel_10688\132693369.py:14: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_chunk = y[start_idx:end_idx]
C:\Users\druzh\AppData\Local\Temp\ipykernel_10688\132693369.py:14: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_chunk = y[start_idx:end_idx]
C:\Users\druzh\AppData\Local\Temp\ipykernel_10688\132693369.py:14: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated

198


C:\Users\druzh\AppData\Local\Temp\ipykernel_10688\132693369.py:14: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_chunk = y[start_idx:end_idx]
C:\Users\druzh\AppData\Local\Temp\ipykernel_10688\132693369.py:14: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_chunk = y[start_idx:end_idx]
C:\Users\druzh\AppData\Local\Temp\ipykernel_10688\132693369.py:14: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated

199


C:\Users\druzh\AppData\Local\Temp\ipykernel_10688\132693369.py:14: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_chunk = y[start_idx:end_idx]
C:\Users\druzh\AppData\Local\Temp\ipykernel_10688\132693369.py:14: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_chunk = y[start_idx:end_idx]
C:\Users\druzh\AppData\Local\Temp\ipykernel_10688\132693369.py:14: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated

In [31]:
import joblib

joblib.dump(model, 'logistic_regression_w_embs_only_model_200_epochs.pkl')

['logistic_regression_w_embs_only_model_200_epochs.pkl']

In [34]:
import random
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
prediction = model.predict(X_test_final)

mse = mean_squared_error(y_test, prediction)
print("Mean Squared Error:", mse)

f1 = f1_score(y_test, prediction)
print("f1:", f1)

accuracy = accuracy_score(y_test, prediction)
precision = precision_score(y_test, prediction)
recall = recall_score(y_test, prediction)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Mean Squared Error: 0.2988843217850851
f1: 0.6298480930833872
Accuracy: 0.7011156782149148
Precision: 0.6928273042396231
Recall: 0.5773646396563218
